In [1]:
import pandas as pd
import numpy as np
import os
from os import listdir
from os.path import isfile, isdir, join
import ast
import sys
import time
import datetime

#vader
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import re
import string
nltk.download('stopwords')
from nltk.corpus import stopwords

#Display Setting
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)

#detect language
from textblob import TextBlob  

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Ace\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ace\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
## Useful functions
def seconds_time(s):
    seconds_set = " "+str(s%60)+"sec" if s%60!=0 else ""
    s = s//60
    
    mins_set = " "+str(s%60)+"min" if s%60!=0 else ""
    s = s//60
    
    hrs_set = " "+str(s%24)+"hr" if s%24!=0 else ""
    s = s//24
    
    s = str(s)+"days" if s!=0 else ""
    
    return s+hrs_set+mins_set+seconds_set

In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

sc = SparkContext("local[2]")
spark = SparkSession.builder.master("local[2]").getOrCreate()

### Contents of Comments data
- author: author name
- author_fullname: author ID
- body: text of comments
- created_utc: datetime the comment was created
- id: comment id
- link_id: submission id that the comment is under
- parent_id: id of the comment that this comment is under
- score: Not sure but keeping just in case its some special score
- subreddit: subreddit name
- subreddit_id: subreddit id
- updated_utc: datetime the comment was last updated

In [4]:
start_time_df = datetime.datetime.now()

#directory of the unzipped data profile
df = spark.read.json(r".\*\*\comments\*.json")

stop_time_df = datetime.datetime.now()
print("Data done extract in "+seconds_time((stop_time_df-start_time_df).seconds)+" with count of "+str(df.count()))

Data done extract in  8min 56sec with count of 735120


In [6]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [7]:
df_set = df.withColumn("created_utc", F.col("created_utc")/1000)
df_set = df_set.withColumn("created_utc", F.from_unixtime("created_utc"))
df_set = df_set.orderBy(F.col("created_utc").asc())
df_set = df_set.dropDuplicates()
df_set = df_set.orderBy(F.col("created_utc").asc())

In [8]:
start_time = datetime.datetime.now()
display(df_set.show())
stop_time = datetime.datetime.now()
print("Data done extract in "+seconds_time((stop_time-start_time).seconds))

+--------------------+---------------+-------------------+--------------------+-------------------+-------+-------+-----------+---------+-------------+-----+--------------+------------+----------------+
|              author|author_fullname|author_fullname_lvl|                body|        created_utc|     id|link_id|link_id_lvl|parent_id|parent_id_lvl|score|     subreddit|subreddit_id|subreddit_id_lvl|
+--------------------+---------------+-------------------+--------------------+-------------------+-------+-------+-----------+---------+-------------+-----+--------------+------------+----------------+
|           baitshop1|       2djs9bnu|                 t2|Just put on some ...|2019-01-01 08:00:01|ecy7pmk| ab67jz|         t3|   ab67jz|           t3|    7|wallstreetbets|       2th52|              t5|
|             ssini92|          b1j9f|                 t2|But still not clo...|2019-01-01 08:00:05|ecy7prw| ab4cnf|         t3|  ecxuhwj|           t1|    1|wallstreetbets|       2th52|   

None

Data done extract in  3min 35sec


In [9]:
df_set.toPandas().to_csv("comments_full.csv", index=False)